<h2>Answering business queries using the Chinook Database</h2>
<h3>Introduction to database:</h3>
<p>The database represents a online music store which includes tables of some fake and some real data for the employees, customers, invoices and the music tracks along with some meta data.</p>
<h3>Database Scheme (Visual):</h3>
![alt text](schema.png "Chinook Schema")
<h3>Import Modules:</h3>

In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

<h3>Functions:</h3>

In [2]:
# return a dataframe of the queried sql results table
def execute_query(query):
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql(query, conn)
    
# return a sql create view result
def execute_command(command):
    with sqlite3.connect('chinook.db') as conn:
        conn.isolation_level = None # auto commits any changes
        conn.execute(command)

# return a list of all tables and views in the database
def show_tables():
    query = '''SELECT name, type 
               FROM sqlite_master 
               WHERE type IN ("table", "view");
            '''
    result = execute_query(query)
    print(result)

<h3>Test show_tables() function</h3>
It should display all the tables and views that exists in the Chinook Database

In [3]:
show_tables()

                  name   type
0                album  table
1               artist  table
2             customer  table
3             employee  table
4                genre  table
5              invoice  table
6         invoice_line  table
7           media_type  table
8             playlist  table
9       playlist_track  table
10               track  table
11  one_sale_countries   view
12     other_countries   view
13        usa_invoices   view
14     usa_genres_sold   view
15         other_group   view


<h3>Questions:</h3>
<p>The following are the DataQuest questions that we have to answer.</p>
<h4>Question 1)</h4>
<p>The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four.  All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:</p>
<table>
<thead>
<tr>
<th>Artist Name</th>
<th>Genre</th>
</tr>
</thead>
<tbody>
<tr>
<td>Regal</td>
<td>Hip-Hop</td>
</tr>
<tr>
<td>Red Tone</td>
<td>Punk</td>
</tr>
<tr>
<td>Meteor and the Girls</td>
<td>Pop</td>
</tr>
<tr>
<td>Slim Jim Bites</td>
<td>Blues</td>
</tr>
</tbody>
</table>
<p>The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.</p>
<p>You'll need to write a query to find out which genres sell the most tracks in the USA, and then create a visualization of that data using pandas.</p>

In [4]:
# drop the view if it exists (if retrying)
command = "DROP VIEW IF EXISTS usa_invoices;"
result = execute_command(command)

# set up view to return all invoices related to the usa
query = '''
        CREATE VIEW usa_invoices AS
            SELECT b.track_id   
            FROM invoice as a, invoice_line as b
            WHERE a.invoice_id = b.invoice_id AND a.billing_country = 'USA';
        '''
result = execute_command(query)
print("Current Tables and Views:")
show_tables()

Current Tables and Views:
                  name   type
0                album  table
1               artist  table
2             customer  table
3             employee  table
4                genre  table
5              invoice  table
6         invoice_line  table
7           media_type  table
8             playlist  table
9       playlist_track  table
10               track  table
11  one_sale_countries   view
12     other_countries   view
13     usa_genres_sold   view
14         other_group   view
15        usa_invoices   view


In [5]:
# drop the view if it exists (if retrying)
command = "DROP VIEW IF EXISTS usa_genres_sold;"
result = execute_command(command)

# use the previous result table to get all the genere ids of each track
# and fetch the genre name
query = '''
        CREATE VIEW usa_genres_sold AS
            SELECT c.name
            FROM usa_invoices as a, track as b, genre as c
            WHERE a.track_id = b.track_id AND b.genre_id = c.genre_id;
        '''
result = execute_command(query)
print("Current Tables and Views:")
show_tables()

Current Tables and Views:
                  name   type
0                album  table
1               artist  table
2             customer  table
3             employee  table
4                genre  table
5              invoice  table
6         invoice_line  table
7           media_type  table
8             playlist  table
9       playlist_track  table
10               track  table
11  one_sale_countries   view
12     other_countries   view
13         other_group   view
14        usa_invoices   view
15     usa_genres_sold   view


In [6]:
# get the aggregate of the number of tracks sold in the USA
# and as a per percentage of all the genres
query = '''
        SELECT name as genre, count(name) as tracks_sold, 
               ROUND(CAST(count(name) AS FLOAT)/
               CAST((SELECT count(name) FROM usa_genres_sold) AS FLOAT), 2)
               as percentage_sold
        FROM usa_genres_sold 
        GROUP BY name
        ORDER BY tracks_sold DESC;
        '''
result = execute_query(query)
print(result)

                 genre  tracks_sold  percentage_sold
0                 Rock          561             0.53
1   Alternative & Punk          130             0.12
2                Metal          124             0.12
3             R&B/Soul           53             0.05
4                Blues           36             0.03
5          Alternative           35             0.03
6                Latin           22             0.02
7                  Pop           22             0.02
8          Hip Hop/Rap           20             0.02
9                 Jazz           14             0.01
10      Easy Listening           13             0.01
11              Reggae            6             0.01
12   Electronica/Dance            5             0.00
13           Classical            4             0.00
14         Heavy Metal            3             0.00
15          Soundtrack            2             0.00
16            TV Shows            1             0.00


<b>Comment:</b> Here, we see that the most popular genres, say top 3, are Rock with 561 tracks sold in the USA followed by Alternative & Punk with 130 sold followed by 124 tracks sold in the Metal genre. Also, we see that Rock has been the main contributor of the genres sold in the USA accounting for 53% of the total USA track sales. This gives a rough idea on what the USA population likes to listen to and purchase from this store. So, it maybe wise to load up on more Rock music artists to increase sales in this music store business.

<h4>Question 2)</h4>
<p>Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

You might like to consider whether any extra columns from the employee table explain any variance you see, or whether the variance might instead be indicative of employee performance.</p>

In [7]:
# Setup: Connect employee.employee_id with customer.support_red_id to connect
#        the customer with the employee that served them. 
#        Then get the invoice for the customer by connecting customer.customer_id
#        with the invoice.customer_id to connect the invoice with the customer
#        and then get the total from the invoice table and join it all up as shown.
query = '''
        SELECT a.employee_id, a.first_name, a.last_name, sum(c.total) as sales
        FROM employee as a, customer as b, invoice as c
        WHERE a.employee_id = b.support_rep_id AND
              b.customer_id = c.customer_id
        GROUP BY a.employee_id, a.first_name, a.last_name
        ORDER BY sales DESC;
        '''
print(execute_query(query))

   employee_id first_name last_name    sales
0            3       Jane   Peacock  1731.51
1            4   Margaret      Park  1584.00
2            5      Steve   Johnson  1393.92


<b>Comment:</b> So, we see that the employee with the most sales is Jane Peacock with \$1731.51 followed by Margaret Park with \$1584 music sales and lastly we have Steve Johnson with \$1393.92. Thus, Jane Peacock is the best performing employee based on the amount of sales along. 

<h4>Question 3)</h4>
<p>Your next task is to analyze the sales data for customers from each different country.  You have been given guidance to use the country value from the <code>customers</code> table, and ignore the country from the billing address in the <code>invoice</code> table.</p>
<p>In particular, you have been directed to calculate data, for each country, on the:</p>
<ul>
<li>total number of customers</li>
<li>total value of sales</li>
<li>average value of sales per customer</li>
</ul>
<p>Because there are a number of countries with only one customer, you should group these customers as "Other" in your analysis.

In [20]:
# drop the view if it exists
command = "DROP VIEW IF EXISTS other_group;"
result = execute_command(command)

# first create a view that gets all countries with 1 customer and consider them
# all under the group 'Other'
command = '''
          CREATE VIEW other_group AS
              SELECT * 
              FROM (SELECT country, count(country) as ncustomers 
                    FROM (SELECT first_name, last_name, country
                          FROM customer
                          ORDER BY country)
                    GROUP BY country)
              WHERE ncustomers = 1;
          '''
result = execute_command(command)

print("Countries with 1 customer:")
print(execute_query("SELECT * FROM other_group"))

Countries with 1 customer:
        country  ncustomers
0     Argentina           1
1     Australia           1
2       Austria           1
3       Belgium           1
4         Chile           1
5       Denmark           1
6       Finland           1
7       Hungary           1
8       Ireland           1
9         Italy           1
10  Netherlands           1
11       Norway           1
12       Poland           1
13        Spain           1
14       Sweden           1


In [47]:
# drop the view if it exists
command = "DROP VIEW IF EXISTS other_sales;"
result = execute_command(command)

# secondly get the sales of the 'Other' group
command = '''
          CREATE VIEW other_sales AS
              SELECT sum(a.ncustomers) as total_customers, sum(c.total) as sales,
                     ROUND(CAST(sum(c.total) AS FLOAT)/CAST(sum(a.ncustomers) AS FLOAT), 2) 
                         AS average_sale_per_customer
              FROM other_group as a, customer as b, invoice as c
              WHERE a.country = b.country AND b.customer_id = c.customer_id;
          '''
result = execute_command(command)

# get the other group results
other_group = execute_query("SELECT * FROM other_sales")
other_group.insert(0, "country", "Other")

In [50]:
# drop the view if it exists
command = "DROP VIEW IF EXISTS main_sale;"
result = execute_command(command)

# thirdly get the total sales by each country along with the 'Other' group sales
command = '''
        CREATE VIEW main_sale AS
            SELECT a.country, count(a.country)as total_customers,
                   sum(b.total) as sales, 
                   ROUND((CAST(sum(b.total) AS FLOAT)/CAST(count(a.country) 
                       AS FLOAT)), 2) as average_sale_per_customer
            FROM customer as a, invoice as b
            WHERE country NOT IN (SELECT country FROM other_group) AND
                  a.customer_id = b.customer_id
            GROUP BY country
            ORDER BY country;
        '''
result = execute_command(command)

# aggregate the sales for each country and total customers and average sales per customer
df = execute_query("SELECT * FROM main_sale")

# append the other group to the main dataframe
df = df.append(other_group)

print("Music Sales ($) per country:\n", df)

Music Sales ($) per country:
           country  total_customers    sales  average_sale_per_customer
0          Brazil               61   427.68                       7.01
1          Canada               76   535.59                       7.05
2  Czech Republic               30   273.24                       9.11
3          France               50   389.07                       7.78
4         Germany               41   334.62                       8.16
5           India               21   183.15                       8.72
6        Portugal               29   185.13                       6.38
7             USA              131  1040.49                       7.94
8  United Kingdom               28   245.52                       8.77
0           Other              147  1094.94                       7.45


<b>Comment:</b> Here, it is clear that the highest sales total and customer count is from the "Other" group. The countries in the "Other" group are Argentina, Australia, Austria, Belgium, Chile, Denmark, Finland, Hungary,        Ireland, Italy, Netherlands, Norway, Poland, Spain and Sweden who all only have 1 customer each in the database. However, since they are all grouped up because of their small sample size we can not conclude that selling to all these countries increases profit. Although, we can conclude that USA as the next best in terms of the amount of customers and total sales. But, the highest country with the best average sale amount per customer belongs to Czech Republic with $9.11.